In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.abstract_quantum_scaling.abstract_quantum_scaling import QScalingSimulation
plt.rcParams['image.cmap'] = 'viridis'

# Focus on the classical case to better understand the incomplete control

In [3]:
plt.close('all')
# initialize
N = 256
out_mode = N // 2
T = 1/np.sqrt(N) * np.random.normal(loc=0, scale=np.sqrt(2)/2, size=(N, N, 2)).view(np.complex128)[:, :, 0]

# back propagate from desired
desired_vec = np.zeros(N, complex)
desired_vec[out_mode] = 1
at_slm = T.transpose() @ desired_vec

# best S
angles_from_out = np.angle(at_slm)
S = -angles_from_out

# incomplete control macro pixels
macro_pixel_size = 2
max_index = (len(at_slm)//macro_pixel_size)*macro_pixel_size

truncated = at_slm[:max_index]
averaged = truncated.reshape((-1, macro_pixel_size)).sum(axis=1)
repeated = np.repeat(averaged, macro_pixel_size)
end_val = at_slm[max_index:].sum()
padded = np.pad(repeated, (0, len(at_slm) % macro_pixel_size), constant_values=(0, end_val))
S_macro_pixels = -np.angle(padded)

# incomplete control zeroing
N_to_remove = round(N * 0.5)
global_phase = np.angle(at_slm[N-N_to_remove:].sum())
S_zero = S.copy()
S_zero[N-N_to_remove:] = global_phase + 0.5

# propagate
in_vec = np.ones(N, complex) / np.sqrt(N)
out_full =         T @ np.diag(np.exp(1j*S))              @ in_vec
out_macro_pixels = T @ np.diag(np.exp(1j*S_macro_pixels)) @ in_vec
out_zero =         T @ np.diag(np.exp(1j*S_zero))         @ in_vec

# plotting
fig, ax = plt.subplots()
ax.plot(np.abs(out_full)**2, label='out full')
ax.plot(np.abs(out_macro_pixels)**2, label='macro pixels')
ax.plot(np.abs(out_zero)**2, label='zero')
ax.axhline(np.pi/16, color='c', linestyle='--')
ax.axhline(np.pi/8, color='b', linestyle='--')
ax.axhline(np.pi/4, color='g', linestyle='--')
ax.set_ylim([0, 1])
ax.set_xlim([N//2 - 10, N//2 + 10])
ax.legend()

## I think the conclusion is that it should give different results

In [4]:
N = 60000
ms = [1, 2, 3, 4, 5, 8, 10, 20]
I_MPXS = []
I_ZROS = []
for m in ms:
    # The 1/sqrt(2N) is because I have 2N number with variance of 1. This means this is a good row of a TM
    # But into this TM should enter light that is 1/sqrt(N)
    X = 1/np.sqrt(N) * 1/np.sqrt(2*N) * np.random.normal(size=(N, 2)).view(np.complex128)

    # Macro pixels
    Y = X.reshape(-1, m).copy()
    Y_macrod = Y.sum(axis=1)
    out_macros = np.abs(Y_macrod).sum()
    I_macro_ps = np.abs(out_macros)**2
    I_MPXS.append(I_macro_ps)

    # Zeroing
    # First 1/m with corrected phases, rest with uncorrected phases
    out_optimized = np.abs( X[:N//m] ).sum()
    out_not_optimized = np.abs( X[N//m:].sum() )
    out_zero = out_optimized + out_not_optimized
    print(f'{m=}: optimized-to-not ratio={out_not_optimized / out_optimized:.3f}')
    I_zeroing = np.abs(out_zero)**2
    I_ZROS.append((I_zeroing))

dummy = np.linspace(0, 1, 100)
y_linear = np.pi/4 * dummy
y_sqr = np.pi/4 * dummy**2

ms = np.array(ms)
_, ax = plt.subplots()
ax.plot(1/ms, I_MPXS, '*', label='macro pixels', color='#1f77b4')
ax.plot(1/ms, I_ZROS, '*', label='zeroing', color='#ff7f0e')
ax.plot(dummy, y_linear, linestyle=':', color='#1f77b4', label='analytic linear')
ax.plot(dummy, y_sqr, linestyle=':', color='#ff7f0e', label='analytic sqr')
ax.set_xlabel('Degree of control (1/m)')
ax.set_ylabel('Intensity (a.u.)')
ax.legend()

### And now I think I even understand this, see under the "thoughts and calculations" file